### Importing the necessary libraries

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')


### Importing the dataset

In [ ]:
df=pd.read_csv('C:\\Users\\ADMIN\\Desktop\\Movie_data\\credits.csv') 
df1=pd.read_csv('C:\\Users\\ADMIN\\Desktop\\Movie_data\\keywords.csv') 
df2=pd.read_csv('C:\\Users\\ADMIN\\Desktop\\Movie_data\\links.csv') 
df3=pd.read_csv('C:\\Users\\ADMIN\\Desktop\\Movie_data\\links_small.csv') 
df4=pd.read_csv('C:\\Users\\ADMIN\\Desktop\\Movie_data\\movies_metadata.csv') 
df5=pd.read_csv('C:\\Users\\ADMIN\\Desktop\\Movie_data\\ratings_small.csv') 
keywords = df1
links = df2
links_small = df3
ratings_small = df5
#data we need 
movies_metadata = df4
movies_metadata = movies_metadata.head(10000)
movies_metadata['index'] = movies_metadata.index
credits = df
credits['id'] = (credits['id']).astype('str')
credits = credits.head(10000)

### Data Exploration

In [2]:
#I'm extracting the feautures needed for building a model. Others are not neccesary.
data = movies_metadata.loc[:,['title','id','genres','original_language','release_date','overview']]
#I'm Converting both id columns into str for the merge operation. Merge operation requires the 2 id
# columns to be of same datatype
data['id'] =data['id'].astype('str')
df_final =  ( pd.merge(left=data, right=credits, left_on='id', right_on='id') )
#df_final is a dataset made with id,genres,cast,crew,release date,language as these are feautures that
#help in content based recommendations
df_final

,title,id,genres,original_language,release_date,overview,cast,crew
0,Toy Story,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,1995-10-30,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,Jumanji,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,1995-12-15,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,Grumpier Old Men,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,1995-12-22,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,Waiting to Exhale,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,1995-12-22,"Cheated on, mistreated and stepped on, the wom...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,Father of the Bride Part II,11862,"[{'id': 35, 'name': 'Comedy'}]",en,1995-02-10,Just when George Banks has recovered from his ...,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...,...,...,...,...
10001,Miracle in Milan,43379,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",it,1951-02-08,Once upon a time an old woman discovers a baby...,"[{'cast_id': 2, 'character': 'La vecchia Lolot...","[{'credit_id': '5712c499c3a368673d004567', 'de..."
10002,Before the Fall,9393,"[{'id': 18, 'name': 'Drama'}]",de,2004-07-04,"In 1942, Friedrich Weimer's boxing skills get ...","[{'cast_id': 1, 'character': 'Friedrich Weimer...","[{'credit_id': '52fe44f2c3a36847f80b356d', 'de..."
10003,The Frisco Kid,16972,"[{'id': 37, 'name': 'Western'}, {'id': 35, 'na...",en,1979-07-06,Rabbi Avram arrives in Philadelphia from Polan...,"[{'cast_id': 1, 'character': 'Avram', 'credit_...","[{'credit_id': '57cb6aee92514163dd003b6e', 'de..."
10004,Onmyoji: The Yin Yang Master,21325,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",ja,2001-10-06,"During a dark time in the Heian period, when e...","[{'cast_id': 3, 'character': 'Fujiwara no Tada...","[{'credit_id': '532116ff9251411f890020a3', 'de..."


### Missing Value treatment

In [3]:
# replacing the null valuess with null string

for feature in df_final:
  df_final[feature] =df_final[feature].fillna('') 
movies_data = df_final

In [4]:
movies_data['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

### Data Preprocessing

In [ ]:
# Cleaning genres dataset, removing useless stuff
import ast
genres_clean = []
cnt = 1
for val in movies_data['genres'] :
    i = 1
    res = ast.literal_eval(val)
    str = ""
    for l in res :
        str = str + l["name"] + " "
        i = i + 1
    genres_clean.append(str)
movies_data["genres"] = genres_clean

In [ ]:
# Cleaning cast dataset, removing useless stuff
import ast
cast_clean = []
for val in movies_data['cast'] :
    i = 1
    res = ast.literal_eval(val)
    str = ""
    for l in res :
       temp = (l["character"])
       x = temp.replace(" (voice)", "", 1)
       str = str + x + " "
    cast_clean.append(str)
movies_data['cast'] = cast_clean

In [9]:
#removing \ns
str = " "
s=[]
for val in movies_data["cast"]:
    without_line_breaks = val.replace("\n", " ")
    s.append(without_line_breaks)
movies_data['cast'] = s


In [10]:
#cleaning crew dataset, removing useless stuff
import ast
crew_clean = []
for val in movies_data['crew'] :
    i = 1
    res = ast.literal_eval(val)
    str = ""
    i=0
    for l in res :
       temp = (l["name"])
       x = temp
       str = str + x + "  "
    crew_clean.append(str)
crew_clean
movies_data['crew'] = crew_clean
movies_data

,title,id,genres,original_language,release_date,overview,cast,crew
0,Toy Story,862,Animation Comedy Family,en,1995-10-30,"Led by Woody, Andy's toys live happily in his ...",Woody Buzz Lightyear Mr. Potato Head Slinky Do...,John Lasseter Joss Whedon Andrew Stanton Jo...
1,Jumanji,8844,Adventure Fantasy Family,en,1995-12-15,When siblings Judy and Peter discover an encha...,Alan Parrish Samuel Alan Parrish / Van Pelt Ju...,Larry J. Franco Jonathan Hensleigh James Hor...
2,Grumpier Old Men,15602,Romance Comedy,en,1995-12-22,A family wedding reignites the ancient feud be...,Max Goldman John Gustafson Ariel Gustafson Mar...,Howard Deutch Mark Steven Johnson Mark Steve...
3,Waiting to Exhale,31357,Comedy Drama Romance,en,1995-12-22,"Cheated on, mistreated and stepped on, the wom...",Savannah 'Vannah' Jackson Bernadine 'Bernie' H...,Forest Whitaker Ronald Bass Ronald Bass Ezr...
4,Father of the Bride Part II,11862,Comedy,en,1995-02-10,Just when George Banks has recovered from his ...,George Banks Nina Banks Franck Eggelhoffer Ann...,Alan Silvestri Elliot Davis Nancy Meyers Na...
...,...,...,...,...,...,...,...,...
10001,Miracle in Milan,43379,Drama Fantasy Comedy,it,1951-02-08,Once upon a time an old woman discovers a baby...,La vecchia Lolotta Totò Rappi Mobbi Edvige Mar...,Eraldo Da Roma Vittorio De Sica Vittorio De ...
10002,Before the Fall,9393,Drama,de,2004-07-04,"In 1942, Friedrich Weimer's boxing skills get ...",Friedrich Weimer Albrecht Stein Christoph Schn...,Torsten Breuer Molly von Fürstenberg Harald ...
10003,The Frisco Kid,16972,Western Comedy,en,1979-07-06,Rabbi Avram arrives in Philadelphia from Polan...,Avram Tommy Mr. Jones Chief Gray Cloud Darryl ...,Terence Marsh Mace Neufeld Marvin March Haw...
10004,Onmyoji: The Yin Yang Master,21325,Drama Fantasy Horror,ja,2001-10-06,"During a dark time in the Heian period, when e...",Fujiwara no Tadako Abe no Seimei Minamoto no H...,Shigeru Umebayashi Yojiro Takita Baku Yumema...


**TfidVectorizer**:

Used TfidfVectorizer() to convert the raw word documents in to TF-IDF feature matrix.

> TF-IDF is an abbreviation for Term Frequency Inverse Document Frequency. This is very common algorithm to transform text into a meaningful representation of numbers which is used to fit machine algorithm for prediction.

It is a measure of originality of a word by comparing the number of times a word appears in a document with the number of documents the word appears in.

TF-IDF = (Number of times word appears in a doc) + $\log(\frac{n}{df})$

where, n -> no of documents and df -> doc freq of the word

****Cosine Similarity****:

Cosine similarity is a metric, that is used to determine the similarity between data objects(vectors) irespective of their size. 

The formula for Cosine similarity = Cos(x,y) = $\frac{x \times y}{||x|| \times ||y||}$

where, 
- $x \times y$ -> dot product of vectors x and y
- ||x|| and ||y|| -> length of vectors
- $||x|| \times ||y||$ -> cross product of vectors

### Predicting movie - **Overview + Genres**

In [11]:
movies_data['index'] = movies_data.index
# combining the overview and genres features
combined_features = movies_data['overview'] + movies_data['genres']
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)
print(similarity) 

[[1.         0.03585524 0.01893887 ... 0.02505088 0.02589314 0.00729217]
 [0.03585524 1.         0.061902   ... 0.03811432 0.04318521 0.05247614]
 [0.01893887 0.061902   1.         ... 0.01961587 0.03893848 0.00747262]
 ...
 [0.02505088 0.03811432 0.01961587 ... 1.         0.024499   0.07864813]
 [0.02589314 0.04318521 0.03893848 ... 0.024499   1.         0.02703932]
 [0.00729217 0.05247614 0.00747262 ... 0.07864813 0.02703932 1.        ]]


****Prediction****

In [17]:
# Take name of the moive as an input from the user
movie_name = input(' Enter your favourite movie name : ')

# listing the movie titles
list_of_all_titles = movies_data['title'].tolist()

# finding close match using difflib library
"""
difflib.get_close_matches function returns a list of close matched strings. The order of the close matched string is based on the similarity score.
"""
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

# getting the similarity scores
similarity_score = list(enumerate(similarity[index_of_the_movie]))

# And sorting the similarity scores in a descending order i.e highest to lowest
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

# Printing the top 30 recommended movies
print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : The karate kid
Movies suggested for you : 

1 . The Karate Kid
2 . The Karate Kid, Part III
3 . The Karate Kid, Part II
4 . It All Starts Today
5 . Tall Tale
6 . Klute
7 . The Devil and Daniel Webster
8 . Forever Young
9 . Android
10 . Mrs. Doubtfire
11 . Master of the Flying Guillotine
12 . In July
13 . The Apartment
14 . Five Fingers of Death
15 . Taxi 3
16 . The Prophecy II
17 . The Next Karate Kid
18 . Sunday Bloody Sunday
19 . The Last Dragon
20 . The Very Thought of You
21 . The Young Master
22 . The Streetfighter
23 . Lone Wolf McQuade
24 . House Party
25 . Once Upon a Time in China
26 . Ashes of Time
27 . The Big Squeeze
28 . Prince of the City
29 . Wisdom


### Predicting movie - **Overview + Genres + Title**

In [16]:
movies_data['index'] = movies_data.index
combined_features = movies_data['overview'] + movies_data['genres'] + movies_data['title']
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)
print(similarity) 

[[1.         0.03401144 0.01782533 ... 0.0249831  0.02328151 0.0060975 ]
 [0.03401144 1.         0.0578492  ... 0.03788557 0.03867603 0.04533453]
 [0.01782533 0.0578492  1.         ... 0.02915631 0.0346942  0.00605191]
 ...
 [0.0249831  0.03788557 0.02915631 ... 1.         0.02677372 0.06915324]
 [0.02328151 0.03867603 0.0346942  ... 0.02677372 1.         0.02196353]
 [0.0060975  0.04533453 0.00605191 ... 0.06915324 0.02196353 1.        ]]


****Prediction****

In [18]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : Batman
Movies suggested for you : 

1 . Batman
2 . Batman Beyond: Return of the Joker
3 . Batman Returns
4 . Batman: Mask of the Phantasm
5 . Batman & Robin
6 . Batman Forever
7 . The Batman Superman Movie: World's Finest
8 . Masterminds
9 . Sword of the Valiant: The Legend of Sir Gawain and the Green Knight
10 . We Were Soldiers
11 . Batman & Mr. Freeze: SubZero
12 . Enemy at the Gates
13 . Last Hurrah for Chivalry
14 . Maximum Overdrive
15 . Enemy Mine
16 . Batman
17 . The Shadow
18 . City By The Sea
19 . Just Visiting
20 . Bring Me the Head of Alfredo Garcia
21 . Ong-Bak: The Thai Warrior
22 . WiseGirls
23 . Alone in the Dark
24 . The Amateur
25 . The War at Home
26 . State Property 2
27 . Underworld
28 . American Gigolo
29 . Highlander
